# Customer satisfaction at the Banks in Saratov, Russia 

**My particular interest in the topic connected with I was going to open a bank account online, but in a while, I could use some service on-site in Saratov city in Russia, so as I am from there originally. 
I have not been in Saratov for almost 3 years and don't know what is going on there. 
So, my task is to choose the bank in Saratov, Russia which better deals with their customers. 

In [1]:
import psycopg2
import pandas as pd

In [2]:
username = 'olga'
password = 'olga_google'
host = '77.244.65.15'

The access to the database was provided by Arthur Semyonov, who parses the data from the websites of different Russian banks. 

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
conn = psycopg2.connect(
        dbname='dwh',
        user=username,
        password=password,
        host=host,
        port=5432)
cursor = conn.cursor()

For the beginning, let us take a look at the database provided:

In [6]:
query = '''
    select 
        id
        , link 
        , title
        , city 
        , bank_name
        , score 
        , status 
        , username 
        , create_dt 
        , comments
    from home.dt_banki_responses
    order by id desc 
    limit 10
'''
cursor.execute(query)
result = cursor.fetchall()

This function helps us to run postgreSQL queries in Python:

In [7]:
def custom_read_from_sql(query, n):
    df = pd.pandas.read_sql_query(query, conn)
    return df.head(n)

Let us restrict ourselves by the data from the current year (2021):

In [8]:
q = """
    select 
        *
    from 
        home.dt_banki_responses 
    where 
        date(create_dt) >= '2021-01-01'
"""
custom_read_from_sql(q, 5)

,id,link,title,city,bank_name,score,status,username,create_dt,comments,content,bank_answer,bank_answer_date,admin_answer,admin_answer_date,parse_dt
0,10567124,https://www.banki.ru/services/responses/bank/r...,Мобильное приложение «Телекард 2.0» не работае...,г. Москва,Газпромбанк,3.0,Проблема решена,A. N.,2021-09-24 11:57:00+00:00,3,Добрый день.Являюсь клиентом Газпромбанка боле...,Здравствуйте . .Начали проверку по вашему вопр...,2021-09-24 16:04:00+00:00,XNA,NaT,2021-10-01
1,10565496,https://www.banki.ru/services/responses/bank/r...,Некорректно работает Телекард 2.0,г. Москва,Газпромбанк,3.0,Проверяется,igorS1966,2021-09-19 20:42:00+00:00,1,Газпромбанк приложение Телекард 2.0 работает у...,"Здравствуйте, Уточним информацию по вашему воп...",2021-09-20 17:01:00+00:00,Спасибо за отзыв. Давайте подождем ответ от ба...,2021-09-27 10:21:00+00:00,2021-10-01
2,10558384,https://www.banki.ru/services/responses/bank/r...,Списание денежных средств,г. Владивосток (Приморский край),Газпромбанк,1.0,Проверяется,Belfegor26,2021-09-02 12:19:00+00:00,3,Обратился в банк в г. Владивосток ул. Океански...,ЗдравствуйтеВзяли ваш вопрос в работу.Проверим...,2021-09-02 17:36:00+00:00,Спасибо за отзыв. Давайте подождм окончательно...,2021-09-13 19:33:00+00:00,2021-10-01
3,10569658,https://www.banki.ru/services/responses/bank/r...,Необоснованный овердрафт,г. Москва,Хоум Кредит Банк,1.0,Без статуса,kirilltish,2021-10-01 20:24:00+00:00,0,Активно пользуюсь дебетовыми картами банка с 2...,Добрый день.Спасибо за отзыв.Хочу извиниться з...,2021-10-04 15:59:00+00:00,XNA,NaT,2021-10-08
4,10569667,https://www.banki.ru/services/responses/bank/r...,"""Лучший"" банк",г. Москва,Тинькофф Банк,1.0,Проверяется,- SAV -,2021-10-01 20:58:00+00:00,0,Здравствуйте!Не могу не поделиться свежими впе...,XNA,NaT,XNA,NaT,2021-10-01


**Customers have the opportunity to leave references after using the bank's service. They put grades 1-5. Let us calculate which bank is better according to the customer response. 
For now, I will not restrict myself to only banks and customers from Saratov, so as the quality of service depends mostly on how business operations are organized in general, 
then on particular work of a local subsidiary.
So, let us look at which banks answer their customers more quickly, which do not answer at all.

In [9]:
q = """
with re as (
    select 
        bank_name as nameb, 
        cast(count(create_dt) as decimal) as f_date,
        cast(count(bank_answer_date) as decimal) as resp_date,
        cast(sum(case when bank_answer_date notnull then 0 else 1 end) as decimal) as no_resp
    from 
        home.dt_banki_responses
    where 
        date(create_dt) >= '2021-01-01'
    group by bank_name
)
select 
    nameb, 
    f_date,
    no_resp,
    resp_date, 
    round(re.no_resp/re.f_date*100,1) || ' %' as perc_no_ans
from re
order by perc_no_ans desc;
"""
custom_read_from_sql(q, 15)

,nameb,f_date,no_resp,resp_date,perc_no_ans
0,Райффайзенбанк,1166.0,94.0,1072.0,8.1 %
1,Альфа-Банк,7153.0,476.0,6677.0,6.7 %
2,Хоум Кредит Банк,5560.0,257.0,5303.0,4.6 %
3,ВТБ,10411.0,360.0,10051.0,3.5 %
4,СберБанк,11404.0,356.0,11048.0,3.1 %
5,Газпромбанк,3025.0,85.0,2940.0,2.8 %
6,Тинькофф Банк,26700.0,686.0,26014.0,2.6 %
7,Почта Банк,2440.0,57.0,2383.0,2.3 %
8,Банк Открытие,3091.0,66.0,3025.0,2.1 %
9,МТС Банк,1225.0,6.0,1219.0,0.5 %


**Raiffeisen Bank, Alpha Bank, and Home Credit bank have the most % of no-answering to customers' inquiries at all. 

**Let us calculate way more important indicator  - the scores that customers give to their banks: 

In [10]:
q = """
with db as
(
    select 
        id,
        bank_name,
        city,
        score,
        create_dt,
        bank_answer_date 
    from 
        home.dt_banki_responses
    where 
        date(create_dt) >= '2021-01-01'
    order by bank_name,city
), ref_number as 
(
    select 
        bank_name,
        count(id) as customer, 
        count(score) as num_ref,
        count(id) - count(score) as no_resp,
        round((count(id) - count(score))/cast(count(id) as decimal)*100,1) ||'  %' as Percent_no_ref
    from db
    group by bank_name
    order by percent_no_ref
)
select 
    bank_name,  
    count(id) as num_transaction, 
    count(score) as Num_score, 
    sum(case when score=5 then 1 else 0 end) as num_of_5,
    sum(case when score=4 then 1 else 0 end) as num_of_4,
    sum(case when score=3 then 1 else 0 end) as num_of_3,
    sum(case when score=2 then 1 else 0 end) as num_of_2,
    sum(case when score=1 then 1 else 0 end) as num_of_1,
    sum(case when score is null then 1 else 0 end) as no_score
from db 
group by bank_name
order by bank_name;
"""

custom_read_from_sql(q, 15)

,bank_name,num_transaction,num_score,num_of_5,num_of_4,num_of_3,num_of_2,num_of_1,no_score
0,Альфа-Банк,7153,5979,343,45,164,597,4830,1174
1,Банк Открытие,3091,2695,605,62,79,274,1675,396
2,ВТБ,10411,9000,2738,132,199,817,5114,1411
3,Газпромбанк,3025,2741,498,68,95,285,1795,284
4,МТС Банк,1225,1104,577,32,10,57,428,121
5,Почта Банк,2440,2064,224,22,37,165,1616,376
6,Райффайзенбанк,1166,970,120,15,29,101,705,196
7,СберБанк,11404,8726,1357,68,208,770,6323,2678
8,Тинькофф Банк,26700,25523,19294,833,300,836,4260,1177
9,Точка,724,702,629,32,7,7,27,22


**It is not very informative in raw numbers, so, let us convert the numbers of different scores into percentage:

In [11]:
q = '''
with saratov as
(
    select 
        id, bank_name, city, score,create_dt,bank_answer_date 
    from home.dth_banki_responses
    where date(create_dt) >= '2021-01-01'
    order by bank_name,city
), ref_number as 
(
    select 
        bank_name,
        count(id) as customer,
        count(score) as num_ref,
        count(id) - count(score) as no_resp,
        round((count(id) - count(score))/cast(count(id) as decimal)*100,1) ||'  %' as Percent_no_ref
    from saratov
    group by bank_name
    order by percent_no_ref
), distr_scores as 
(
select 
    bank_name,  
    count(id) as num_transaction, 
    count(score) as Num_score, 
    sum(case when score=5 then 1 else 0 end) as num_of_5,
    sum(case when score=4 then 1 else 0 end) as num_of_4,
    sum(case when score=3 then 1 else 0 end) as num_of_3,
    sum(case when score=2 then 1 else 0 end) as num_of_2,
    sum(case when score=1 then 1 else 0 end) as num_of_1,
    sum(case when score is null then 1 else 0 end) as no_score
from saratov
group by bank_name
)
select 
    bank_name,
    num_transaction,
    num_score,
    round(num_of_5/cast(num_transaction as decimal)*100,1) || ' %' as percent_of_5,
    round(num_of_4/cast(num_transaction as decimal)*100,1) || ' %' as percent_of_4,
    round(num_of_3/cast(num_transaction as decimal)*100,1) || ' %' as percent_of_3,
    round(num_of_2/cast(num_transaction as decimal)*100,1) || ' %' as percent_of_2,
    round(num_of_1/cast(num_transaction as decimal)*100,1) || ' %' as percent_of_1,
    round(no_score/cast(num_transaction as decimal)*100,1) || ' %' as percent_no_score,
    num_of_5/cast(num_transaction as decimal)*100 + 
    num_of_4/cast(num_transaction as decimal)*100 + 
    num_of_3/cast(num_transaction as decimal)*100 + 
    num_of_2/cast(num_transaction as decimal)*100 + 
    num_of_1/cast(num_transaction as decimal)*100 + 
    no_score/cast(num_transaction as decimal)*100 as total_perc
from distr_scores
order by num_of_5/cast(num_transaction as decimal) desc;
'''


custom_read_from_sql(q, 15)


,bank_name,num_transaction,num_score,percent_of_5,percent_of_4,percent_of_3,percent_of_2,percent_of_1,percent_no_score,total_perc
0,Точка,789,767,87.1 %,4.6 %,0.9 %,0.9 %,3.8 %,2.8 %,100.0
1,Тинькофф Банк,30789,29501,69.5 %,2.9 %,1.3 %,3.4 %,18.7 %,4.2 %,100.0
2,Хоум Кредит Банк,6829,6490,50.3 %,10.1 %,2.4 %,4.8 %,27.4 %,5.0 %,100.0
3,МТС Банк,1225,1104,47.1 %,2.6 %,0.8 %,4.7 %,34.9 %,9.9 %,100.0
4,ВТБ,14247,12621,22.4 %,1.1 %,2.0 %,8.8 %,54.4 %,11.4 %,100.0
5,Банк Открытие,4057,3622,16.9 %,1.7 %,2.6 %,9.7 %,58.4 %,10.7 %,100.0
6,Газпромбанк,4234,3898,14.4 %,1.8 %,4.6 %,10.4 %,61.0 %,7.9 %,100.0
7,СберБанк,11404,8726,11.9 %,0.6 %,1.8 %,6.8 %,55.4 %,23.5 %,100.0
8,Райффайзенбанк,1475,1270,9.1 %,1.2 %,2.3 %,8.9 %,64.6 %,13.9 %,100.0
9,Почта Банк,2683,2294,8.9 %,0.9 %,1.5 %,6.9 %,67.4 %,14.5 %,100.0


**Tinkoff Bank and Home Credit Bank, and MTS Bank seem the best ones in terms of customers' satisfaction. 
Tochka bank shows the best result in terms of the ratio of good grades to a number of references, but the number of transactions is low, so we do not take this bank into consideration. 

**Now, when we know banks with the best clients' references nationwide, let see what is going on in Saratov: 

In [12]:
q = '''
with saratov as
(
    select 
        id, bank_name, city, score,create_dt,bank_answer_date 
    from home.dth_banki_responses
    where city like '%Сарат%'
    order by bank_name,city
), ref_number as 
(
    select 
        bank_name,
        count(id) as customer,
        count(score) as num_ref,
        count(id) - count(score) as no_resp,
        round((count(id) - count(score))/cast(count(id) as decimal)*100,1) ||'  %' as Percent_no_ref
    from saratov
    group by bank_name
    order by percent_no_ref
), distr_scores as 
(
    select 
        bank_name,  
        count(id) as num_transaction, 
        count(score) as Num_score, 
        sum(case when score=5 then 1 else 0 end) as num_of_5,
        sum(case when score=4 then 1 else 0 end) as num_of_4,
        sum(case when score=3 then 1 else 0 end) as num_of_3,
        sum(case when score=2 then 1 else 0 end) as num_of_2,
        sum(case when score=1 then 1 else 0 end) as num_of_1,
        sum(case when score is null then 1 else 0 end) as no_score
    from saratov
    group by bank_name
)
select 
    bank_name,
    num_transaction,
    num_score,
    round(num_of_5/cast(num_transaction as decimal)*100,1) || ' %' as percent_of_5,
    round(num_of_4/cast(num_transaction as decimal)*100,1) || ' %' as percent_of_4,
    round(num_of_3/cast(num_transaction as decimal)*100,1) || ' %' as percent_of_3,
    round(num_of_2/cast(num_transaction as decimal)*100,1) || ' %' as percent_of_2,
    round(num_of_1/cast(num_transaction as decimal)*100,1) || ' %' as percent_of_1,
    round(no_score/cast(num_transaction as decimal)*100,1) || ' %' as percent_no_score,
    num_of_5/cast(num_transaction as decimal)*100 + 
    num_of_4/cast(num_transaction as decimal)*100 + 
    num_of_3/cast(num_transaction as decimal)*100 + 
    num_of_2/cast(num_transaction as decimal)*100 + 
    num_of_1/cast(num_transaction as decimal)*100 + 
    no_score/cast(num_transaction as decimal)*100  as total_perc
from distr_scores
order by num_of_5/cast(num_transaction as decimal) desc;
'''
custom_read_from_sql(q, 15)


,bank_name,num_transaction,num_score,percent_of_5,percent_of_4,percent_of_3,percent_of_2,percent_of_1,percent_no_score,total_perc
0,МТС Банк,18,18,66.7 %,5.6 %,0.0 %,5.6 %,22.2 %,0.0 %,100.0
1,Тинькофф Банк,393,370,65.6 %,3.1 %,1.8 %,5.1 %,18.6 %,5.9 %,100.0
2,Точка,11,10,63.6 %,18.2 %,0.0 %,9.1 %,0.0 %,9.1 %,100.0
3,Хоум Кредит Банк,360,265,29.7 %,2.8 %,3.6 %,6.1 %,31.4 %,26.4 %,100.0
4,Банк Открытие,328,250,18.6 %,1.8 %,3.4 %,9.1 %,43.3 %,23.8 %,100.0
5,СберБанк,526,386,13.1 %,0.8 %,1.3 %,6.7 %,51.5 %,26.6 %,100.0
6,Альфа-Банк,375,269,12.5 %,0.8 %,1.9 %,8.5 %,48.0 %,28.3 %,100.0
7,Газпромбанк,133,116,9.8 %,3.0 %,0.8 %,14.3 %,59.4 %,12.8 %,100.0
8,ВТБ,314,259,8.0 %,0.3 %,3.5 %,9.9 %,60.8 %,17.5 %,100.0
9,Райффайзенбанк,54,38,3.7 %,0.0 %,1.9 %,16.7 %,48.1 %,29.6 %,100.0


**The leaders again are MTC Bank and Tinkoff Bank, but Tochka has a  small number of references, so we will not take it into the account.  

**There is information about date of client's inquery and when bank answered on it. Let's calculate the average time to answer the customers, if the answer has been at all. 

In [13]:
q = ''' 
with saratov as (
    select 
        id, bank_name, city, score,create_dt,bank_answer_date 
    from home.dt_banki_responses
    where city like '%Сара%' 
    and date(create_dt) >= '2021-01-01'
    order by bank_name, city
)
, time_to_answer as ( 
    select 
        bank_name,
        create_dt,
        bank_answer_date,
        bank_answer_date - create_dt as time_for_response,
        extract(epoch from bank_answer_date - create_dt) as w8_seconds,
        extract(epoch from bank_answer_date - create_dt) / 60 as w8_minutes,
        extract(epoch from bank_answer_date - create_dt) / 60 / 60 as w8_hours,
        extract(epoch from bank_answer_date - create_dt) / 60 / 60 / 24 as w8_days
    from saratov
    where bank_answer_date is not null 
    order by bank_name
)
select 
    bank_name,
    count(*) as num_requests,
    sum(w8_hours),
    round(avg(w8_hours::numeric), 1)    || ' hours' as avg_w8_hours,
    round(median(w8_hours::numeric), 1) || ' hours' as median_w8_hours,
    round(avg(w8_days::numeric), 1)     || ' days'  as avg_w8_days,
    round(median(w8_days::numeric), 1)  || ' days'  as median_w8_days
from time_to_answer
group by bank_name
order by avg(w8_hours::numeric) asc;
'''

custom_read_from_sql(q, 15)

,bank_name,num_requests,sum,avg_w8_hours,median_w8_hours,avg_w8_days,median_w8_days
0,СберБанк,142,645.166667,4.5 hours,2.4 hours,0.2 days,0.1 days
1,МТС Банк,21,215.700000,10.3 hours,10.6 hours,0.4 days,0.4 days
2,Газпромбанк,38,421.450000,11.1 hours,7.0 hours,0.5 days,0.3 days
3,Банк Открытие,33,1057.933333,32.1 hours,11.5 hours,1.3 days,0.5 days
4,ВТБ,74,2983.150000,40.3 hours,24.1 hours,1.7 days,1.0 days
5,Хоум Кредит Банк,98,11152.850000,113.8 hours,79.0 hours,4.7 days,3.3 days
6,Альфа-Банк,61,8821.566667,144.6 hours,139.5 hours,6.0 days,5.8 days
7,Тинькофф Банк,356,71850.433333,201.8 hours,97.5 hours,8.4 days,4.1 days
8,Почта Банк,37,9016.316667,243.7 hours,116.6 hours,10.2 days,4.9 days
9,Райффайзенбанк,11,2868.016667,260.7 hours,145.1 hours,10.9 days,6.0 days


**The absolute leader here is the oldest and biggest bank in Russia Sberbank, but MTC Bank, which already shows itself as one of the best in terms of client satisfaction, is in the second place. 

**Let us calculate the status mentioned on the website after each inquiry "Problem solved", "Done", "Not done", "Without status":

In [14]:
q = """
    select 
        distinct *, round(cast(status_cnt as numeric) / full_cnt * 100, 1) || ' %' as prcnt
    from (
        select 
            bank_name 
            , status 
            , count(*) over(partition by bank_name, status) as status_cnt
            , count(*) over(partition by bank_name) as full_cnt
        from 
            home.dt_banki_responses
            where city like '%Сара%' and
            date(create_dt) >= '2021-01-01'
        order by bank_name
    ) as t1
    order by bank_name, status
"""
custom_read_from_sql(q, 20)

,bank_name,status,status_cnt,full_cnt,prcnt
0,Альфа-Банк,Без статуса,19,67,28.4 %
1,Альфа-Банк,Зачтено,10,67,14.9 %
2,Альфа-Банк,Не засчитана,25,67,37.3 %
3,Альфа-Банк,Проблема решена,1,67,1.5 %
4,Альфа-Банк,Проверяется,12,67,17.9 %
5,Банк Открытие,Без статуса,14,34,41.2 %
6,Банк Открытие,Зачтено,3,34,8.8 %
7,Банк Открытие,Не засчитана,12,34,35.3 %
8,Банк Открытие,Проблема решена,4,34,11.8 %
9,Банк Открытие,Проверяется,1,34,2.9 %


**I don't know the difference between "Problem solved" and "Done". It seems both as positive results. But this query might be useful when we want to check a particular bank. 

# Summary

**This investigation helped me to choose a bank in Saratov while I was physically present in the USA for several years. 
I had already known that Sberbank, Raffaizenbak, Alpha Bank, and Tinkoff Bank are well known as good banks, but this study helped me understand what is going on from the customer satisfaction perspective. 
In spite of the national leader Sberbank answering all their clients most quickly from all other banks, the clients are much more satisfied with Tinkoff bank (69% positive feedbacks against 14% at Sberbank). 

**I checked the Tinkoff website after this study, they offer really good products in terms of interest rate and % for transferring funds. So, it looks like a good choice.   



# References

1) How to run SQL quiries from Python: https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html
2) The access to database was provided by Arthur Semenov who parses data from banki.ru https://artydev.ru